```
#This is formatted as code


```

### **Install Ludwig and Ludwig's LLM related dependencies.**

We Install Ludwig from the latest release

In [1]:
# !pip uninstall -y tensorflow --quiet
# !pip install ludwig
# !pip install ludwig[llm]

Install Ludwig from Ludwig github master

In [2]:
# !pip uninstall -y tensorflow --quiet
# !pip install git+https://github.com/ludwig-ai/ludwig.git@master --quiet
# !pip install "git+https://github.com/ludwig-ai/ludwig.git@master#egg=ludwig[llm]" --quiet

### **Setup Your HuggingFace Token**

Obtain a [HuggingFace API Token](https://huggingface.co/settings/tokens).

In [2]:
import locale; locale.getpreferredencoding = lambda: "UTF-8"
import logging
import yaml
import os
import torch
import getpass



os.environ["HUGGING_FACE_HUB_TOKEN"] = getpass.getpass("Token:")
assert os.environ["HUGGING_FACE_HUB_TOKEN"]

Token:··········


In [3]:
import numpy as np; np.random.seed(123)
import pandas as pd

df = pd.read_json("https://raw.githubusercontent.com/sahil280114/codealpaca/master/data/code_alpaca_20k.json")

# We're going to create a new column called `split` where:
# 90% will be assigned a value of 0 -> train set
# 5% will be assigned a value of 1 -> validation set
# 5% will be assigned a value of 2 -> test set
# Calculate the number of rows for each split value
total_rows = len(df)
split_0_count = int(total_rows * 0.9)
split_1_count = int(total_rows * 0.05)
split_2_count = total_rows - split_0_count - split_1_count

# Create an array with split values based on the counts
split_values = np.concatenate([
    np.zeros(split_0_count),
    np.ones(split_1_count),
    np.full(split_2_count, 2)
])

# Shuffle the array to ensure randomness
np.random.shuffle(split_values)

# Add the 'split' column to the DataFrame
df['split'] = split_values
df['split'] = df['split'].astype(int)

# We can use 2000 rows for this project
df = df.head(n=2000)

## **Understanding The Code Alpaca Dataset** 📖


This Code Alpaca dataset will be used to train a large language model (Llama-2). The finetuned model should basically follow instructions to produce code from natural language. Each row in the dataset consists of an:
- `instruction` that explain a task,
- `input` when additional context is required for the instruction and,
- the expected `output` which is the label.


- ## **Some analysis on the data**

In [4]:
without_more_context = (df['input'] == '').sum()
with_more_context = df.shape[0] - without_more_context

# using 2000 rows
print(f"Total number of examples in the dataset: {df.shape[0]}")

print(f"% of examples that are self-sufficient: {without_more_context/df.shape[0] * 100}")
print(f"% of examples that are need additional context: {with_more_context/df.shape[0] * 100}")
print(f"Counts of each split's category: {df['split'].value_counts()}")

Total number of examples in the dataset: 2000
% of examples that are self-sufficient: 47.0
% of examples that are need additional context: 53.0
Counts of each split's category: 0    1810
2      97
1      93
Name: split, dtype: int64


## **Prompt Engineering**

**Here we use qlora configuration to fine-tune the pre-trained model**

In [6]:
from ludwig.api import LudwigModel

# Specify the path to your YAML configuration file
qlora_config_file_path = 'qlora_fine_tuning_config.yaml'

model = LudwigModel(config=qlora_config_file_path, logging_level=logging.INFO)
results = model.train(dataset=df)

INFO:ludwig.utils.print_utils:
INFO:ludwig.utils.print_utils:╒════════════════════════╕
INFO:ludwig.utils.print_utils:│ EXPERIMENT DESCRIPTION │
INFO:ludwig.utils.print_utils:╘════════════════════════╛
INFO:ludwig.utils.print_utils:
INFO:ludwig.api:╒══════════════════╤═════════════════════════════════════════════════════════════════════════════════════════╕
│ Experiment name  │ api_experiment                                                                          │
├──────────────────┼─────────────────────────────────────────────────────────────────────────────────────────┤
│ Model name       │ run                                                                                     │
├──────────────────┼─────────────────────────────────────────────────────────────────────────────────────────┤
│ Output directory │ /content/results/api_experiment_run                                                     │
├──────────────────┼─────────────────────────────────────────────────────────────────

INFO:ludwig.utils.tokenizers:Loaded HuggingFace implementation of meta-llama/Llama-2-7b-hf tokenizer
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.
INFO:ludwig.features.text_feature:Max length of feature 'None': 220 (without start and stop symbols)
INFO:ludwig.features.text_feature:Setting max length using dataset: 222 (including start and stop symbols)
INFO:ludwig.features.text_feature:max sequence length is 222 for feature 'None'
INFO:ludwig.utils.tokenizers:Loaded HuggingFace implementation of meta-llama/Llama-2-7b-hf tokenizer
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.
INFO:ludwig.features.text_feature:Max length of feature 'output': 819 (without start and stop symbols)
INFO:ludwig.features.text_feature:Setting max length using dataset: 821 (including start and stop symbols)
INFO:ludwig.featur

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

INFO:ludwig.models.llm:Done.
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:362: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.1` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(
INFO:ludwig.utils.tokenizers:Loaded HuggingFace implementation of meta-llama/Llama-2-7b-hf tokenizer
INFO:ludwig.models.llm:==================================================
INFO:ludwig.models.llm:Trainable Parameter Summary For Fine-Tuning
INFO:ludwig.models.llm:Fine-tuning with adapter: lora
INFO:ludwig.models.llm:==================================================
INFO:ludwig.utils.print_utils:
INFO:ludwig.utils.print_utils:╒══════════╕
INFO:ludwig.utils.print_utils:│ TRAINING │
INFO:ludwig.util

trainable params: 4,194,304 || all params: 6,742,609,920 || trainable%: 0.06220594176090199


...

INFO:ludwig.data.batcher.random_access:Last batch in epoch only has 1 sample and will be dropped.


Training: 100%|██████████| 2000/2000 [21:48<00:00,  1.36it/s, loss=0.0216]

INFO:ludwig.trainers.trainer:
Running evaluation for step: 2000, epoch: 1
INFO:ludwig.trainers.trainer:Evaluation took 0.1285s

INFO:ludwig.utils.metrics_printed_table:╒═══════════════════════╤════════════╤══════════════╤════════╕
│                       │      train │ validation   │ test   │
╞═══════════════════════╪════════════╪══════════════╪════════╡
│ bleu                  │     0.3943 │              │        │
├───────────────────────┼────────────┼──────────────┼────────┤
│ char_error_rate       │     0.7601 │              │        │
├───────────────────────┼────────────┼──────────────┼────────┤
│ loss                  │     0.3452 │              │        │
├───────────────────────┼────────────┼──────────────┼────────┤
│ next_token_perplexity │ 14095.6064 │              │        │
├───────────────────────┼────────────┼──────────────┼────────┤
│ perplexity            │ 31114.2090 │              │        │
├───────────────────────┼────────────┼──────────────┼────────┤
│ rouge1_fmea

Training: 100%|██████████| 2000/2000 [21:48<00:00,  1.53it/s, loss=0.0216]

INFO:ludwig.utils.print_utils:
INFO:ludwig.utils.print_utils:╒══════════╕
INFO:ludwig.utils.print_utils:│ FINISHED │
INFO:ludwig.utils.print_utils:╘══════════╛
INFO:ludwig.utils.print_utils:


#**Inference**

In [7]:
test_examples = pd.DataFrame([
      {
            "instruction": "Create an array of length 7 containing all even numbers between 1 and 14.",
            "input": ''
      },
      {
            "instruction": "Generate the square root of 4",
            "input": "",
      },
      {
            "instruction": "Generate an array of length 12 with numbers that are divisible by 4 up to 48",
            "input": ""
      },
      {
            "instruction": "Generate a function that print all the missing data from a dataset",
            "input": "",
      },
      {
            "instruction": "Write a nested loop to print every combination of numbers between 0 and 7.",
            "input": "",
      },

      {
            "instruction": "Create a function that finds the maximum number in a given list.",
            "input": "",
      },
      {
            "instruction": "Design a class to store employee names, ages, and salaries.",
            "input": "",
      },
      {
            "instruction": "Create an array of length 6 containing all odd numbers between 1 and 11",
            "input": "",
      },
      {
            "instruction": "Generate an array of length 10 with numbers that are divisible by 5 up to 50.",
            "input": "",
      },
      {
            "instruction": "Write a nested loop to print every combination of letters 'A' to 'C' and numbers 1 to 3.",
            "input": "",
      },
      {
            "instruction": "Create a function called kunle_average that calculates the average of numbers in a given list.",
            "input": "",
      },
      {
            "instruction": "Print out the values in the following dictionary.",
            "input": "my_dict = {\n  'name': 'Sarah',\n  'age': 25,\n  'city': 'Plano'\n}",
            },
      ])

predictions = model.predict(test_examples)[0]
for input_with_prediction in zip(test_examples['instruction'], test_examples['input'], predictions['output_response']):
  print(f"Instruction: {input_with_prediction[0]}")
  print(f"Input: {input_with_prediction[1]}")
  print(f"Generated Output: {input_with_prediction[2][0]}")
  print("\n\n")

INFO:ludwig.utils.tokenizers:Loaded HuggingFace implementation of meta-llama/Llama-2-7b-hf tokenizer
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


Prediction:   0%|          | 0/1 [00:00<?, ?it/s]

INFO:ludwig.models.llm:For generating text, using: GenerationConfig {
  "max_length": 32,
  "max_new_tokens": 512,
  "temperature": 0.1,
  "transformers_version": "4.33.2"
}

INFO:ludwig.models.llm:Decoded text inputs for the first example in batch: below is an instruction that describes a task, paired with an input that provides further context. write a response that appropriately completes the request.
### instruction: create an array of length 7 containing all even numbers between 1 and 14.
### input: 
### response:
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:362: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.1` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
INFO:ludwig.models.llm:Decoded generated output for the first example in batch: below is an instruction that describes a task, paired with an input that provides furth

Prediction: 100%|██████████| 1/1 [00:42<00:00, 42.34s/it]


INFO:ludwig.utils.tokenizers:Loaded HuggingFace implementation of meta-llama/Llama-2-7b-hf tokenizer


Instruction: Create an array of length 7 containing all even numbers between 1 and 14.
Input: 
Generated Output: array = [2, 4, 6, 8, 10, 12, 14]



Instruction: Generate the square root of 4
Input: 
Generated Output: 2



Instruction: Generate an array of length 12 with numbers that are divisible by 4 up to 48
Input: 
Generated Output: array = [4, 8, 12, 16, 20, 24, 28, 32, 36, 40, 44, 48]



Instruction: Generate a function that print all the missing data from a dataset
Input: 
Generated Output: def print_missing_data(dataset):
    for key, value in dataset.items():
        if value == 0:
            print(key)



Instruction: Write a nested loop to print every combination of numbers between 0 and 7.
Input: 
Generated Output: for i in range(0, 8):
    for j in range(0, 8):
        print(i, j)



Instruction: Create a function that finds the maximum number in a given list.
Input: 
Generated Output: def find_max(list):
    max_num = list[0]
    for num in list:
        if num > max_num

/usr/local/lib/python3.10/dist-packages/ludwig/features/feature_utils.py:102: RuntimeWarning: divide by zero encountered in log
  return np.sum(np.log(sequence_probabilities))
